## 3. WAV-JSON 파일명 매칭 검증

In [1]:
import os
import json

data_dir = "/home/ubuntu/data"  # 데이터가 있는 상위 폴더
folder_structure = {}
json_info_by_folder = {}
wav_files_by_folder = {}

# 전체 폴더 탐색
for root, _, files in os.walk(data_dir):
    folder_name = root.replace(data_dir, "").strip(os.sep) or "root"

    # 폴더 내 파일 1~2개만 저장 (기존 코드 유지)
    folder_structure[folder_name] = files[:2]  

    # 폴더별로 WAV 파일 리스트 초기화
    if folder_name not in wav_files_by_folder:
        wav_files_by_folder[folder_name] = []

    # 폴더별로 JSON 파일 리스트 초기화
    if folder_name not in json_info_by_folder:
        json_info_by_folder[folder_name] = []

    # WAV 파일 정보 저장
    for file in files:
        if file.endswith(".wav"):
            wav_files_by_folder[folder_name].append((file, root))  # (파일명, 폴더 경로)

    # JSON 파일 내부 데이터 추출
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    file_name = data.get("audio", {}).get("fileName", "")
                    labels = [anno.get("labelName", "") for anno in data.get("annotations", [])]

                    json_info_by_folder[folder_name].append({
                        "json_file": file,
                        "fileName": file_name, 
                        "labels": labels, 
                        "json_path": root
                    })
            except Exception as e:
                print(f"❌ {file_path} JSON 파일 읽기 오류: {e}")

# 결과 출력 (폴더별로 최대 1개의 파일 정보만 출력)
print("\n📂 폴더별 파일 리스트:")
for folder, files in folder_structure.items():
    print(f"{folder}: {files}")

print("\n🎵 폴더별 WAV 파일 목록 (각 폴더 최대 1개):")
for folder, wav_list in wav_files_by_folder.items():
    if wav_list:
        wav_file, path = wav_list[0]
        print(f"{folder}: {wav_file} ({path})")
    # WAV 파일이 없는 폴더는 출력하지 않음

print("\n📄 폴더별 JSON 파일 내 정보 (각 폴더 최대 1개):")
for folder, json_list in json_info_by_folder.items():
    if json_list:
        info = json_list[0]
        print(f"{folder}: {info['json_file']}, fileName: {info['fileName']}, labels: {info['labels']}, 경로: {info['json_path']}")
    # JSON 파일이 없는 폴더는 출력하지 않음


📂 폴더별 파일 리스트:
root: []
test_data: []
test_data/label_data_test: []
test_data/label_data_test/2.Motorcycle: []
test_data/label_data_test/2.Motorcycle/4.horn_of_motorcycle: ['2.motorcycle_horn_39554.json', '2.motorcycle_horn_83873.json']
test_data/label_data_test/2.Motorcycle/5.driving_sound_of_motorcycle: ['2.motorcycle_driving_sound_87075.json', '2.motorcycle_driving_sound_15526.json']
test_data/label_data_test/1.Car: []
test_data/label_data_test/1.Car/2.siren_of_car: ['1.car_siren_34389.json', '1.car_siren_70948.json']
test_data/label_data_test/1.Car/3.driving_sound_of_car: ['1.car_driving_sound_12350.json', '1.car_driving_sound_95483.json']
test_data/label_data_test/1.Car/1.horn_of_car: ['1.car_horn_87318.json', '1.car_horn_87759.json']
test_data/raw_data_test: []
test_data/raw_data_test/2.Motorcycle: []
test_data/raw_data_test/2.Motorcycle/4.horn_of_motorcycle: ['2.motorcycle_horn_85080_1.wav', '2.motorcycle_horn_35046_1.wav']
test_data/raw_data_test/2.Motorcycle/5.driving_sound_of

> 참고

| JSON 파일 위치 | WAV 파일 위치 | JSON 실제 파일명 | WAV 실제 파일명 | JSON 내 파일명 | JSON 내 라벨명 |
|---|---|---|---|---|---|
| `test_data/label_data_test/2.Motorcycle/4.horn_of_motorcycle` | `test_data/raw_data_test/2.Motorcycle/4.horn_of_motorcycle` | `2.motorcycle_horn_39554.json` | `2.motorcycle_horn_85080_1.wav` | `2.이륜자동차_39554.wav` | `2.이륜자동차_39554_1.wav` |
| `test_data/label_data_test/2.Motorcycle/5.driving_sound_of_motorcycle` | `test_data/raw_data_test/2.Motorcycle/5.driving_sound_of_motorcycle` | `2.motorcycle_driving_sound_87075.json` | `2.motorcycle_driving_sound_86149_1.wav` | `2.이륜자동차_87075.wav` | `2.이륜자동차_87075_1.wav` |
| `test_data/label_data_test/1.Car/2.siren_of_car` | `test_data/raw_data_test/1.Car/2.siren_of_car` | `1.car_siren_34389.json` | `1.car_siren_71389_1.wav` | `1.자동차_34389.wav` | `1.자동차_34389_1.wav` |
| `test_data/label_data_test/1.Car/3.driving_sound_of_car` | `test_data/raw_data_test/1.Car/3.driving_sound_of_car` | `1.car_driving_sound_12350.json` | `1.car_driving_sound_12271_1.wav` | `1.자동차_12350.wav` | `1.자동차_12350_1.wav` |
| `test_data/label_data_test/1.Car/1.horn_of_car` | `test_data/raw_data_test/1.Car/1.horn_of_car` | `1.car_horn_87318.json` | `1.car_horn_163_1.wav` | `1.자동차_87318.wav` | `1.자동차_87318_1.wav` |
| `label_data/2.Motorcycle/4.horn_of_motorcycle` | `raw_data/2.Motorcycle/4.horn_of_motorcycle` | `2.motorcycle_horn_62872.json` | `2.motorcycle_horn_83164_1.wav` | `2.이륜자동차_62872.wav` | `2.이륜자동차_62872_1.wav` |
| `label_data/2.Motorcycle/5.driving_sound_of_motorcycle` | `raw_data/2.Motorcycle/5.driving_sound_of_motorcycle` | `2.motorcycle_driving_sound_86881.json` | `2.Motorcycle_driving_sound_14738_1.wav` | `2.이륜자동차_86881.wav` | `2.이륜자동차_86881_1.wav` |
| `label_data/1.Car/2.siren_of_car` | `raw_data/1.Car/2.siren_of_car` | `1.car_siren_11756.json` | `1.car_siren_493_1.wav` | `1.자동차_11756.wav` | `1.자동차_11756_1.wav` |
| `label_data/1.Car/3.driving_sound_of_car` | `raw_data/1.Car/3.driving_sound_of_car` | `1.car_driving_sound_95404.json` | `1.car_driving_sound_1142_1.wav` | `1.자동차_95404.wav` | `1.자동차_95404_1.wav` |
| `label_data/1.Car/1.horn_of_car` | `raw_data/1.Car/1.horn_of_car` | `1.car_horn_10971.json` | `1.car_horn_11131_1.wav` | `1.자동차_10971.wav` | `1.자동차_10971_1.wav` |

In [2]:
import os
import re
import json
from collections import defaultdict

# 한글-영어 매칭 딕셔너리 보완
kor_eng_dict = {
    "자동차": "car",
    "이륜자동차": "motorcycle",
    "경적": "horn",
    "사이렌": "siren",
    "주행음": "driving_sound",
    # 필요한 모든 매핑 추가
}

# 파일명 정규화 함수 수정
def normalize_filename(filename):
    # 확장자 제거
    name, ext = os.path.splitext(filename)
    
    # 대소문자 변환 및 공백 제거
    name = name.lower().replace(' ', '')
    
    # 한글-영어 변환 적용
    for kor, eng in kor_eng_dict.items():
        name = name.replace(kor, eng)
    
    # 정규식으로 고유 ID 추출
    match = re.match(r".*?_(\d+)(?:_\d+)?$", name)
    if match:
        file_id = match.group(1)
    else:
        file_id = None
    
    return name, ext, file_id

# 매칭 검사 함수 수정
def find_mismatched_files(directory):
    wav_files = {}
    json_files = {}

    # 모든 폴더 순회하며 파일 수집
    for root, _, files in os.walk(directory):
        for file in files:
            name, ext, file_id = normalize_filename(file)
            if file_id is None:
                continue  # ID가 없는 파일은 패스
            file_path = os.path.join(root, file)
            if ext == ".wav":
                wav_files[file_id] = file_path
            elif ext == ".json":
                json_files[file_id] = file_path

    # 매칭되지 않은 파일 찾기
    mismatched_wav = set(wav_files.keys()) - set(json_files.keys())
    mismatched_json = set(json_files.keys()) - set(wav_files.keys())

    return mismatched_wav, mismatched_json, wav_files, json_files

# 실행
directory_path = "/home/ubuntu/data"
mismatched_wav, mismatched_json, wav_files, json_files = find_mismatched_files(directory_path)

# 불일치 출력
if mismatched_wav or mismatched_json:
    print("\n🔴 매칭되지 않은 파일 목록:")

    if mismatched_wav:
        print("\n🎵 WAV 파일에 대응하는 JSON 파일이 없습니다:")
        for file_id in mismatched_wav:
            print(f"  - WAV 파일 ID: {file_id}, 경로: {wav_files[file_id]}")

    if mismatched_json:
        print("\n📄 JSON 파일에 대응하는 WAV 파일이 없습니다:")
        for file_id in mismatched_json:
            print(f"  - JSON 파일 ID: {file_id}, 경로: {json_files[file_id]}")
else:
    print("✅ 모든 파일이 매칭되었습니다!")

✅ 모든 파일이 매칭되었습니다!


In [3]:
import os
import json

def validate_json(json_path):
    """JSON 파일을 읽고 내부 데이터 검증"""
    if not os.path.exists(json_path):
        print(f"❌ 파일 없음: {json_path}")
        return False, False, "(파일 없음)", [], "없음"

    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        print(f"\n📂 JSON 파일 로드 성공: {json_path}")  # 디버깅
        print(f"🔑 JSON Keys: {data.keys()}")  # JSON 키 확인

        # JSON 내부 필드 확인
        json_filename = data.get("audio", {}).get("fileName", "").strip()
        annotation_labels = [ann["labelName"] for ann in data.get("annotations", [])]

        if not json_filename:
            print(f"⚠️ 'fileName' 없음: {json_path}")
        if not annotation_labels:
            print(f"⚠️ 'annotations' 없음: {json_path}")

        filename_match = bool(json_filename)
        labels_valid = bool(annotation_labels)

        return filename_match, labels_valid, json_filename, annotation_labels

    except json.JSONDecodeError:
        print(f"❌ JSON 형식 오류: {json_path}")
        return False, False, "(JSON 형식 오류)", [], "없음"
    except UnicodeDecodeError:
        print(f"❌ 인코딩 오류: {json_path}")
        return False, False, "(인코딩 오류)", [], "없음"

# 실행
directory_path = "/home/ubuntu/data"
invalid_json_files = []
json_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith(".json")]

for json_path in json_files:
    filename_match, labels_valid, json_filename, json_labels = validate_json(json_path)

    if not filename_match or not labels_valid:
        invalid_json_files.append((json_path, json_filename, json_labels))

# 오류 JSON 파일 출력 (최대 10개만 표시)
print(f"\n🚨 JSON 내부 데이터 오류 (총 {len(invalid_json_files)}개)")
for i, (json_path, json_filename, json_labels) in enumerate(invalid_json_files[:10]):
    print(f"  - JSON 경로: {json_path}")
    print(f"    ❌ 'fileName': {json_filename}")
    print(f"    🏷 'annotations': {json_labels if json_labels else '(비어 있음)'}")


🚨 JSON 내부 데이터 오류 (총 0개)


In [4]:
import os
import json
import re

# 📝 한글과 영어 매핑 사전
korean_english_map = {
    '이륜자동차': 'motorcycle',
    '자동차': 'car',
    '경적': 'horn',
    '주행음': 'drivingsound',
    '사이렌': 'siren',
    # 필요한 모든 매핑을 추가하세요
}

def normalize_filename(filename):
    if not filename:
        return ''
    original_filename = filename  # 디버깅용

    # 1️⃣ 확장자 제거 (.wav 또는 .json)
    filename_no_ext = re.sub(r'\.(wav|json)$', '', filename)

    # 2️⃣ '.wav' 또는 '.json' 바로 앞의 '_숫자' 제거
    filename_no_suffix = re.sub(r'_\d+$', '', filename_no_ext)

    # 3️⃣ 한글을 영어로 변환
    for kor, eng in korean_english_map.items():
        filename_no_suffix = filename_no_suffix.replace(kor, eng)
    
    # 4️⃣ 공백 및 특수문자 제거, 소문자 변환
    filename_normalized = re.sub(r'[\s]', '', filename_no_suffix).lower()

    # 5️⃣ 대분류와 식별번호 추출
    match = re.match(r'^(2\.motorcycle|1\.car).*?(\d+)$', filename_normalized)
    if match:
        category = match.group(1)  # 대분류 추출
        identifier = match.group(2)  # 식별번호 추출
        normalized_name = f"{category}_{identifier}"
    else:
        normalized_name = filename_normalized  # 매칭되지 않으면 전체를 사용

    # 디버깅 출력 (필요 시 활성화)
    # print(f"정규화 전: '{original_filename}' → 정규화 후: '{normalized_name}'")

    return normalized_name

# 📂 검사할 데이터의 루트 디렉터리
base_dir = "/home/ubuntu/data"

# 🚀 WAV 파일 매핑 생성 (normalized_name을 키로 사용)
wav_file_mapping = {}

for root, _, files in os.walk(base_dir):
    for f in files:
        if f.endswith(".wav"):
            normalized_name = normalize_filename(f)
            wav_file_mapping[normalized_name] = os.path.join(root, f)

# 🚀 모든 폴더를 순회하며 JSON & WAV 비교
unmatched_files = []

for root, _, files in os.walk(base_dir):
    json_files = [f for f in files if f.endswith(".json")]

    for json_file in json_files:
        json_path = os.path.join(root, json_file)
        
        # 📌 JSON 파일명 정규화
        normalized_json_filename = normalize_filename(json_file)
        
        # 📌 JSON 파일 로드
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # 📌 JSON 내의 fileName 및 labelName 정규화
        json_fileName = normalize_filename(data.get("audio", {}).get("fileName", ""))
        labelNames = [normalize_filename(annotation.get("labelName", "")) for annotation in data.get("annotations", [])]
        
        # 📌 모든 이름 수집
        names_to_match = [normalized_json_filename, json_fileName] + labelNames
        
        # 빈 문자열 제거
        names_to_match = [name for name in names_to_match if name]
        
        # 유일한 이름 집합 생성
        unique_names = set(names_to_match)
        
        # WAV 파일 존재 여부 확인
        wav_exists = any(name in wav_file_mapping for name in unique_names)
        
        if not wav_exists:
            unmatched_files.append({
                "json_path": json_path,
                "names_to_match": unique_names,
                "wav_files_found": {name: wav_file_mapping.get(name, '찾을 수 없음') for name in unique_names},
            })

# 🚨 결과 출력
if unmatched_files:
    print(f"\n🚨 매칭되지 않은 파일 발견 (총 {len(unmatched_files)}개)")
    for item in unmatched_files[:10]:  # 최대 10개만 출력
        print(f"\n- JSON 파일 경로: {item['json_path']}")
        print(f"  📌 비교된 이름들 (대분류_식별번호): {list(item['names_to_match'])}")
        print(f"  🔎 WAV 파일 검색 결과:")
        for name, path in item['wav_files_found'].items():
            print(f"    - '{name}': {path}")
else:
    print("🎉 모든 파일이 정상적으로 매칭되었습니다!")

🎉 모든 파일이 정상적으로 매칭되었습니다!
